Unfortunately Github doesn't render ipywidgets, which allow us to interactively alter the parameters of the model. If you want to mess with the parameters yourself then download this file, and also install and activate ipywidgets, which you can do in the command line interface with the following commands:
```
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
jupyter labextension install @jupyter-widgets/jupyterlab-manager
```

<img src="SUschema.png">
Fig 1: Schematic to represent the various reactions of photosynthesis through synthesising units. Each synthesizing unit has incoming and outgoing fluxes. 

<img src = "carbo_oxo.png">
Fig 2: Schematic for simultaneous oxygenation and carboxylation of RuBP according to SU kinetics, whereby forward reactions are much stronger than reverse reactions. 

<h2>Current Model and Next Steps</h2>
The schematic in figure 1 depicts different fluxes for ATP and NADPH. This will likely be neccessary to account for balancing of the ATP:NADPH ratio through Cyclic Electron Transport, export of NADPH, or some other method. Currently, however, we set $j_{ATP} = j_{NADPH} = j_{PQ}$, the rate of photochemcial quenching.

<h3>PGA reduction and Calvin Cycle</h3>
Based on the schematic in figure 1, the net rate of photosynthesis, $j_{prod}$ incorporates the electron transport flux $j_{PQ}$ and the PGA synthesis flux $j_{pga,net}$ in parallel. Formulated according to Kooijman 1998's Synthesizing Unit kinetics:
\begin{equation}
    j_{prod} = (\frac{1}{k_{E_C}} + \frac{1}{\kappa_C j_{PQ}} + \frac{1}{j_{pga,net}} - \frac{1}{\kappa_C j_{PQ} + j_{pga,net}})^{-1}
\end{equation}
where $k_{E_R}$ represents the maximum velocity of enzymes in the reduction of PGA, and $\kappa_C$ represents the number of PGA produced per product of electron transport.
$j_{pga,net} = 2j_{cA} + \frac{3}{2}j_{oA}$ as for every Carbon consumed, 2 PGA are produced, and for every 2 Oxygen consumed, 3 PGA are produced.

We can derive $j_{cA}$ as $-d[C]_T/dt$ based on figure 2, which yields 
$$j_{cA} = -k^+_c[C][E]_T(1 + \frac{k^+_R[R]}{k^+_o[O] + k^+_c[C]})f_E $$
where 
$$ f_E = \frac{[E]}{[E]_T} = (1 + (\frac{1}{k^+_R[R]} + 1)(\tilde{k^+_o}[O] +  \tilde{k^+_c}[C]) + k^+_R[R]\frac{1 + \tilde{k^+_o}[O] +  \tilde{k^+_c}[C]}{k^+_o[O] +  k^+_c[C]})^{-1} $$
where $\tilde{k^+_o} = k^+_o/\tilde{k_{ERO}}$ and $\tilde{k^+_c} = k^+_c/\tilde{k_{ERC}}$, whereby $\tilde{k_{ERC}} = k_{ERC}+k^-_c+k^-_R$ and $\tilde{k_{ERO}} = k_{ERO}+k^-_o+k^-_R$
<br>
By symmetry,
$$j_{oA} = -k^+_o[O][E]_T(1 + \frac{k^+_R[R]}{k^+_c[C] + k^+_o[O]})f_E $$

<h4>Additional factors to consider for Calvin Cycle/PGA reduction.</h4>
PGA and Ribulose-5-Phosphate compete for ATP, whereas in this model the rate of ATP delivered to both reactions (see RuBP regeneration) is equal to the overall rate of electron transport.

<h4>Incorporating RuBP regeneration</h4>
Based on figures 1 and 2, we can derive a dynamical system for the consumption and regeneration of $R$:
$$
\frac{dR}{dt} = -[E]_Tf_E(k^+_R[R] + k^+_c[C] + k^+_o[O]) + (\frac{1}{k_{E_R}} +\frac{1}{\kappa_R j_{PQ}} + \frac{1}{j_{prod}} - \frac{1}{(\kappa_R j_{PQ} + j_{prod}})^{-1}
$$
where $\kappa_R$ represents the number of RuBP generated per product of electron transport, and $k_{E_R} represents the velocities of enzymes involved in RuBP regeneration, such as Ribulose-5-phosphate kinase.  


<h3>Light reactions and electron transport</h3>

<h4>Ye model</h4>
We started by using the electron transport rate derived by Ye et al (Ye 2013):
$$ j_{ph} = \alpha_e \frac{1-\beta_eI}{1+\gamma_eI}I
$$
where $\alpha_e$ is the initial slope of the $j_{ph}$-$I$ curve, and $\beta_e$ and $\gamma_e$ are the coefficiencts for photoinhibition and light saturation, respectively. $I$ is irradiance. This model runs into a couple issues, one being that $j_{ph}$ can become negative, and the other that the probabilities of NPQ and flourescence are independant of light.

<h4> Alternatives to Ye model </h4>

<h4>Model A: Incorporating light dependant flourescence</h4>
We can potentially correct the Ye model by revising the original derivation and multiplying $\xi_3R_{ki}$ by $I$, leading to:
$$
j_{ph} = \alpha_e \frac{1-\frac{\beta_eI}{z_1I+z_2}}{1+\gamma_eI}I
$$


<h4>Model B: Circuit formulation</h4>
We can treat both light dependant ($R_I$) and light independant ($R_C$) photoinhibitory mechanisms as resistances to photochemistry, where $R_I = R_{I0}I$. $I$ is analogous to voltage. 
<img src = "circuit.png">
Thus, using Ohm's law, ${j_ph} = \frac{I}{R_{I0}I+R_C}$. At high light intensities, $j_{ph}$ saturates to $\frac{1}{R_{I0}}$ so we can say $R_{I0} = 1/j_{max}$

<h3> Model C: Balancing fluxes with a dynamical system </h3>
<img src = "lightreaction.png">
We seek to mechanistically model the light reactions by balancing incoming and outgoing fluxes. Light ($j_I$) is intercepted by flourescence ($j_F$) and thermal dissipation ($j_{NPQ}$), the latter of which is controlled by the pH gradient, which is determined by the difference between the incoming flux from photochemical quenching, $j_{PQ}(t-1)$, and outgoing ATP flux, $j_{ATP}(t-1)$. $j_{ATP}$ depends on the rate of the Calvin Cycle, which in turn depends on $j_{PQ}$. Thus, we can formulate $j_{PQ}$ as a discrete dynamical system, where $j_{PQ}(t) = f(j_{PQ}(t-1))$.
Ultimately, it would be interesting if the system is able to imitate the following experimental results:
<img src="darkacc.png">

We require initial conditions for $j_{PQ}(0)$, $j_{ATP}(0)$, and $[R](0)$. 

We treat ATP synthase as a synthesizing unit, where light and ADP from the Calvin cycle are consumed to form ATP. Thus $j_{ATP}$ iterates as such:
$$
j_{ATP}(t) = (\frac{1}{j_{syn}} + \frac{1}{\kappa_A j_{PQ}(t-1)} + \frac{1}{j_{ADP}(t)} - \frac{1}{\kappa_A j_{PQ}(t-1) + j_{ADP}(t)})^{-1}
$$
Where $j_{syn}$ is the velocity of ATP synthase (which is potentially tunable as a regulator of pH), and $\kappa_A$ is the amount of ATP produced per electron excited. The incoming ADP flux $j_{ADP}$ depends on $j_{prod}$ and $j_{RuBP}$, which each have a 1-to-1 correspondance with ADP formation. 
$$
j_{ADP}(t) = j_{prod}(t) + j_{RuBP}(t)
$$
Where $j_{prod}$ is the output flux of a synthesizing unit which processing incoming $j_{ATP}$ and $j_{pga}([R])$:
$$
j_{prod} = (\frac{1}{k_p} + \frac{1}{j_{pga}([R])} + \frac{1}{j_{ATP}} - \frac{1}{j_{ATP} + j_{pga}([R])})^{-1}
$$
And $j_{pga}$ is the same as described earlier.
Finally, $j_{PQ}$ iterates as a function of the partitioning between reaction pathways:
$$
j_{PQ}(t) = \frac{k_{PQ}}{k_{PQ}+k_{F}+k_{NPQ}f(j_{PQ}(t-1))}j_I
$$
Where $k_{PQ}$, $k_{NPQ}$, and $k_{F}$ are rate constants for different quenching pathways, and $f(j_PQ(t-1))$ models pH. As pH increases, more Zeaxanthin and Antheraxin will be available for NPQ. pH depends on proton buildup, which depends on the difference between incoming $j_{PQ}$ and outgoing $j_{ATP}$ (at some point we should also likely consider ATP synthase conductance).
$$
f(j_{PQ}(t-1)) = m(j_{PQ}(t-1) - j_{ATP}(t))
$$


<h4>Potential improvements for light model</h4>
An important variable to include is the conductance of ATP synthase, which appears to change to regulate lumen pH.
<br>
NPQ should probably be able to induce oscillations in photosynthesis, as depicted in the following image.
<img src = "oscillations.png">
We also likely need to find a way to partition $j_{PQ}$ into $j_{ATP}$ and $j_{NADPH}$, incorporating cyclic electron transport (CET) and export of NADPH.
